In [99]:
# Team members and dataset
# Dataset: earthquakes
# MohamedAymane, Sfouli (team head)
# Ryan, Jabbour (team member)
# Jeremie, FormeyDeSaintLouvent (team member)
# Camille, DupreLaTour (team member)

In [100]:
# Import libraries
import pandas as pd
import plotly.express as px
from sklearn.cluster import DBSCAN
from shapely.geometry import Point
import geopandas as gpd
from sklearn.preprocessing import KBinsDiscretizer
from statsmodels.tsa.arima.model import ARIMA

# Dash imports for the final dashboard
from dash import Dash, html, dcc

In [101]:
# Function: load_data
# Input: file_path (str) - the path to the CSV file
# Output: df (pd.DataFrame) - a dataframe containing the loaded data
def load_data(file_path):
    df = pd.read_csv(file_path) # Load the CSV file into a pandas dataframe
    return df # Return the dataframe

In [102]:
# Function: explore_data_shape
# Input: df (pd.DataFrame)
# Output: None
def explore_data_shape(df):
    print("Number of rows:", df.shape[0]) # Number of rows in the dataset
    print("Number of columns:", df.shape[1]) # Number of columns in the dataset

In [103]:
# Function: explore_data_types
# Input: df (pd.DataFrame)
# Output: None
def explore_data_types(df):
    print("\nData types of each column:\n", df.dtypes) # Type of the data contained in each column of the dataset

In [104]:
# Function: explore_data_nb_missing_values
# Input: df (pd.DataFrame)
# Output: None
def explore_data_nb_missing_values(df):
    print("\nNumber of missing values per column:\n", df.isna().sum()) # Number of missing values in each column

In [105]:
# Function: explore_data
# Input: df (pd.DataFrame)
# Output: None
def explore_data_descriptive_stats(df):
    print("\nBasic descriptive statistics for each column:\n", df.describe(include='all')) # Description of all numerical and categorical columns

Indicator 1 :

This indicator analyzes the maximum earthquake magnitudes by century. It groups the data by century, calculates the highest magnitude for each period, and then visualizes the results in a bar chart. The chart highlights the centuries with the most powerful earthquakes, helping to identify trends and periods of significant seismic activity.

In [106]:
# Function: max_magnitude_by_century
# Input: df (pd.DataFrame)
# Output: result_df (pd.DataFrame) with columns ['Century', 'Max_Magnitude']
def max_magnitude_by_century(df):
    df['Century'] = (df['Year'] // 100) * 100 # Define each century
                                              # For BC years (which are negative), centuries will also be negative multiples of 100
    result_df = df.groupby('Century', as_index=False)['Mag'].max().rename(columns={'Mag':'Max_Magnitude'}) # Group the earthquakes by century and find the max magnitude
    return result_df

In [107]:
# Function: visualize_max_magnitude_by_century
# Input: df (pd.DataFrame)
# Output: fig (plotly.graph_objs.Figure)
def visualize_max_magnitude_by_century(result_df):
    fig = px.bar( # Create a bar plot for visualizing the first indicator (maximum magnitude by century)
        result_df, # Data
        x="Century", # Define the x axis as the century
        y="Max_Magnitude", # Define the y axis as the maximum magnitude
        title="Maximum Earthquake Magnitude by Century", # Title of the plot
        labels={"Century": "Century", "Max_Magnitude": "Maximum Magnitude"}, # Label the axes
        text="Max_Magnitude", # Display the magnitude on the bars
        color="Max_Magnitude",  # Use the magnitude for coloring
        color_continuous_scale="Viridis", # Palette of colors
    )
    fig.update_traces(texttemplate='%{text:.1f}', textposition="outside") # Display the magnitude on the bars
    fig.update_layout( 
        xaxis=dict(title="Century"), # Label the x axis
        yaxis=dict(title="Maximum Magnitude", range=[0, 10]), # Label the y axis
        coloraxis_colorbar=dict(title="Magnitude Importance"), # Label the color bar
    )
    return fig

Indicator 2 :

The second indcator takes earthquake magnitude data and makes it easier to understand. First, it adjusts the magnitudes so they’re on a similar scale by standardizing them, meaning it changes the numbers so that they all have a mean of 0 and a standard deviation of 1. After that, it groups the magnitudes into three simple categories: Low, Medium, and High. This makes it easier for someone to quickly see how strong an earthquake was without dealing with complicated numbers.

In [108]:
# Function: standarize_cluster_magnitude
# Input: df (pd.dataframe)
# Output: result_df (pd.dataframe)
def standarize_cluster_magnitude(df):
    mean_mag = df['Mag'].mean() # Calculate the mean of the 'Mag' column
    std_mag = df['Mag'].std() # Calculate the standard deviation of the 'Mag' column
    df['Mag_standarized'] = (df['Mag'] - mean_mag) / std_mag # Z-score normalization for the 'Mag' column

    discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')      # Discretize the 'Mag_zscore' column into 3 categorical bins (low, medium, high)
    df['Mag_discretized'] = discretizer.fit_transform(df[['Mag_standarized']]).astype(int)   # using KBinsDiscretizer
    # Map discretized values to human-readable categories
    bin_labels = {0: 'Low', 1: 'Medium', 2: 'High'}
    df['Mag_discretized'] = df['Mag_discretized'].map(bin_labels)

    return df

In [109]:
# Function: visualize_standarize_cluster_magnitude
# Input: df (pd.dataframe)
# Output: fig (plotly.graph_objs.Figure)
def visualize_standarize_cluster_magnitude(df):
    # Count the occurrences of each discretized category
    category_counts = df['Mag_discretized'].value_counts().reset_index()
    category_counts.columns = ['Magnitude Category', 'Count']
    fig = px.bar( # Create a bar plot for visualizing the distribution of magnitude categories
        category_counts, # Data
        x='Magnitude Category', # Define the x axis as the magnitude category
        y='Count', # Define the y axis as the count
        text='Count', # Display the count on the bars
        title='Distribution of Magnitude Categories', # Title of the plot
        labels={'Count': 'Number of Earthquakes', 'Magnitude Category': 'Category'}, # Label the axes
        color='Magnitude Category', # Use the magnitude category for coloring
        color_discrete_sequence=px.colors.qualitative.Vivid # Palette of colors
    )
    fig.update_traces(texttemplate='%{text}', textposition='outside') # Display the count on the bars
    fig.update_layout( 
        xaxis_title='Magnitude Category', # Label the x axis
        yaxis_title='Count', # Label the y axis
        title_font_size=18, # Set the title font size
        xaxis_tickangle=-45, # Rotate the x axis labels
        width=800, # Width of the plot
        height=600 # Height of the plot
    )
    return fig

The plot of the first indicator shows lack of information in previous centuries which is normal when data wasn't really collected back then. Moreover, we can see that we are having higher maximum earthquake magnitudes nowadays than before.

Indicator 3:

This section introduces an indicator that uses time series forecasting to predict the number of earthquakes in future years. By fitting an ARIMA model to the historical yearly counts of earthquakes, we can estimate what the earthquake frequency might look like in the coming years.

In [110]:
def indicator_temporal_forecast(df):
    """
    Create a temporal indicator by forecasting the future number of earthquakes.
    Input: df (DataFrame)
    Output: forecast_df (DataFrame) - a dataframe with future year and forecasted value
    """
    yearly_counts = df.groupby('Year').size().reset_index(name='COUNT')
    yearly_counts = yearly_counts.sort_values('Year')
    model = ARIMA(yearly_counts['COUNT'], order=(1,1,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=5)
    future_years = range(int(yearly_counts['Year'].max())+1, int(yearly_counts['Year'].max())+6)
    forecast_df = pd.DataFrame({'Year': future_years, 'FORECAST_COUNT': forecast.values})
    return forecast_df

In [117]:
# Create a temporal indicator by forecasting the future number of earthquakes and visualize it.# 
# Parameters:
# df (DataFrame): Input dataframe with 'Year' column for time and counts# 
# Returns:
# None: Displays the forecast visualization
def plot_indicator_temporal_forecast(forecast_df):
    fig_cluster = px.line(forecast_df, x='Year', y='FORECAST_COUNT', title='Temporal Forecast')
    return fig_cluster

Indicator 4:

Our last indicator groups earthquake locations into clusters to identify areas with a high frequency of earthquakes, or "hot-spots." It first removes any rows with missing latitude or longitude values, then uses a method called DBSCAN to group nearby earthquake locations together based on their geographical proximity. It adds a new column to the data to indicate which cluster each earthquake belongs to. The function also prepares the data for mapping by converting it into a GeoDataFrame, making it easy to visualize the clusters on a world map.

In [112]:
# Function: visualize_cluster_earthquake
# Input: df (pd.dataframe)
# Output: result_df (pd.dataframe) with column 'Cluster_label'
def cluster_earthquakes(df):
    result_df = df.dropna(subset=['Latitude', 'Longitude']).copy() # Drop rows with missing latitude or longitude values
    coords = result_df[['Latitude', 'Longitude']].to_numpy() # Extract latitude and longitude columns as a numpy array
    db = DBSCAN(eps=2, min_samples=10).fit(coords) # Perform DBSCAN clustering with epsilon=2 and min_samples=10
    result_df['Cluster_label'] = db.labels_ # Assign cluster labels to each earthquake

    gdf = gpd.GeoDataFrame( # Create GeoDataFrame for visualization
        result_df, geometry=[Point(xy) for xy in zip(result_df['Longitude'], result_df['Latitude'])],
        crs="EPSG:4326"
    )
    countries = gpd.read_file('countries_shp/ne_110m_admin_0_countries.shp') # Import countries shapefile for visualization

    return result_df

In [113]:
# Function: visualize_cluster_earthquakes
# Input: df (pd.dataframe)
# Output: fig (plotly.graph_objs.Figure)
def visualize_cluster_earthquakes(result_df):
    fig = px.scatter_geo( # Create a scatter plot on a world map for visualizing the clusters
        result_df, # Data
        lat='Latitude', # Latitude column
        lon='Longitude', # Longitude column
        color='Cluster_label', # Color the points by cluster label
        title='Spatial Clustering of Earthquakes on World Map', # Title of the plot
        projection='natural earth', # Map projection
        width=1200, # Width of the plot
        height=800 # Height of the plot
    )
    return fig

The plot of the last indicator displays a map of all earthquakes happened between 2150 BC and 2020, clustering them to show earthquakes' fissures

In [118]:
# Main block
if __name__ == "__main__":
    # Step 1: Data collection and exploration
    file_path = "earthquakes.csv"
    data = load_data(file_path)

    explore_data_shape(data)

    explore_data_types(data)

    explore_data_nb_missing_values(data)

    explore_data_descriptive_stats(data)

    # Step 2: Indicators construction

    # Indicator 1: Grouping query indicator
    grouped_result = max_magnitude_by_century(data)

    # Indicator 2: Data transformation
    data = standarize_cluster_magnitude(data)

    # Indicator 3: Temporal indicator (e.g., forecasting)
    forecast_df = indicator_temporal_forecast(data)

    # Indicator 4: Spatial indicator (e.g., spatial clustering)
    gdf_clusters = cluster_earthquakes(data)

    # Step 3: Visual representations
    fig_indicator1 = visualize_max_magnitude_by_century(grouped_result)
    fig_indicator1.show()

    fig_indicator2 = visualize_standarize_cluster_magnitude(data)
    # This is a matplotlib figure, so we can show it using plt.show() if not in a notebook that auto-shows.
    fig_indicator2.show()

    fig_indicator3 = plot_indicator_temporal_forecast(forecast_df)
    fig_indicator3.show()
    
    fig_indicator4 = visualize_cluster_earthquakes(gdf_clusters)
    fig_indicator4.show()

    # Step 4: Visualization Dashboard using Dash

    app = Dash(__name__)
    
    # For demonstration purposes, we won't run the server here. We'll just create the layout.
    app.layout = html.Div([
        html.H1("Earthquakes Dashboard"),
        html.H2("Mohamed Aymane, Sfouli (team head); Ryan, Jabbour (team member); Jeremie, Formey De Saint Louvent (team member); Camille, Dupre La Tour (team member)"),
        html.H3("Dataset: Earthquakes"),
        html.Div([
            html.H4("Indicator 1: Max Magnitude per Centry"),
            dcc.Graph(figure=fig_indicator1)
        ]),
        html.Div([
            html.H4("Indicator 2: Normalized Magnitude Distribution"),
            dcc.Graph(figure=fig_indicator2) 
        ]),
        html.Div([
            html.H4("Indicator 3: Earthquake Count Forecast"),
            dcc.Graph(figure=fig_indicator3)
        ]),
        html.Div([
            html.H4("Indicator 4: Spatial Clustering of Earthquakes"),
            dcc.Graph(figure=fig_indicator4)
        ])
    ])
    # run the Dash server
    app.run_server(debug=True)

Number of rows: 4424
Number of columns: 38

Data types of each column:
 Year                                  float64
Mo                                    float64
Dy                                    float64
Hr                                    float64
Mn                                    float64
Sec                                   float64
Tsu                                   float64
Vol                                   float64
Location Name                          object
Latitude                              float64
Longitude                             float64
Focal Depth (km)                      float64
Mag                                   float64
MMI Int                               float64
Deaths                                float64
Death Description                     float64
Missing                               float64
Missing Description                   float64
Injuries                              float64
Injuries Description                  float64
Damage (